In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

**University Details are obtained from https://www.nirfindia.org/2020/UniversityRanking.html**

In [2]:
# The webpage downloaded to local machine to avoid multiple hit on website

university_url = """/Users/deegee/Desktop/WorkArea/GitHub/indian-colleges/MHRD, National Institute Ranking Framework (NIRF).htm"""


university_url = """./MHRD, National Institute Ranking Framework (NIRF).htm"""

soup = BeautifulSoup(open(university_url), "html.parser")



Reading from website

In [3]:
My_table = soup.find('table',{'id':'tbl_overall'})

Eventhough, we are fetching **Top 100** University, each university is having 2 table to it, resulting in 300 tables in the webpage

In [4]:
table_rows = My_table.find_all('tr')
print("Number of rows {}".format(len(table_rows)))

Number of rows 301


In [5]:
# end_row = 298
# start_row = 1

In [6]:
data_list = []
for i in range(1,301,3):
    data_list.append(table_rows[i])

In [7]:
table_rows_td = table_rows[1].find_all('td')

In [8]:
data_list = []
for i in table_rows_td:
    data_list.append(i.text)
print(data_list)

['IR-O-U-0220', 'Indian Institute of ScienceMore DetailsClose |  | TLR (100)RPC (100)GO (100)OI (100)PERCEPTION (100)82.2292.1682.8952.84100.00', '82.22', '92.16', '82.89', '52.84', '100.00', 'Bengaluru', 'Karnataka', '84.18', '1']


In [9]:
data_list = []
for rows in table_rows:
    """pass each row with td"""
    a = rows.find_all('td')

In [10]:
def extract_univ(univ_text):
    splitted_data = []
    for pos, elem in enumerate(univ_text):
        if pos == 1:
            split_value = elem.split('|')[0]
            splitted_data.append(split_value[0:-18])
        else:
            splitted_data.append(elem)

    return splitted_data

In [11]:
data_dict = {}

for rows in table_rows:
    data_list = []
    row = rows.find_all('td')
    if len(row)>0:
        for element in row:
            data_list.append(element.text)
        if '|' in str(data_list[1]):
            
            data_dict[len(data_dict)] = extract_univ(data_list)    
            
print("Total number of rows ",len(data_dict))            

Total number of rows  100


In [12]:
df = pd.DataFrame(data_dict.values())
df

,0,1,2,3,4,5,6,7,8,9,10
0,IR-O-U-0220,Indian Institute of Science,82.22,92.16,82.89,52.84,100.00,Bengaluru,Karnataka,84.18,1
1,IR-O-U-0109,Jawaharlal Nehru University,74.98,44.80,100.00,75.10,67.24,New Delhi,Delhi,70.16,2
2,IR-O-U-0500,Banaras Hindu University,70.91,44.89,85.65,58.76,53.99,Varanasi,Uttar Pradesh,63.15,3
3,IR-O-U-0436,Amrita Vishwa Vidyapeetham,69.65,52.11,73.66,65.83,44.33,Coimbatore,Tamil Nadu,62.27,4
4,IR-O-U-0575,Jadavpur University,53.38,52.96,90.39,48.67,71.44,Kolkata,West Bengal,61.99,5
...,...,...,...,...,...,...,...,...,...,...,...
95,IR-O-U-0366,Utkal University,42.89,11.50,77.12,44.13,39.00,Bhubaneswar,Odisha,40.05,96
96,IR-O-U-0555,Graphic Era University,55.59,8.36,61.02,59.78,26.73,Dehradun,Uttarakhand,40.04,97
97,IR-O-U-0451,Chettinad Academy of Research and Education,70.68,5.16,48.45,56.63,18.74,Kanchipuram District,Tamil Nadu,39.97,98
98,IR-O-U-0164,Kurukshetra University,49.89,9.95,68.15,48.88,33.21,Kurukshetra,Haryana,39.79,99


In [13]:
selected_df = df[[1,7,8,9,10]]
selected_df.columns = ["University","City","State","Score","Ranking"]
selected_df

,University,City,State,Score,Ranking
0,Indian Institute of Science,Bengaluru,Karnataka,84.18,1
1,Jawaharlal Nehru University,New Delhi,Delhi,70.16,2
2,Banaras Hindu University,Varanasi,Uttar Pradesh,63.15,3
3,Amrita Vishwa Vidyapeetham,Coimbatore,Tamil Nadu,62.27,4
4,Jadavpur University,Kolkata,West Bengal,61.99,5
...,...,...,...,...,...
95,Utkal University,Bhubaneswar,Odisha,40.05,96
96,Graphic Era University,Dehradun,Uttarakhand,40.04,97
97,Chettinad Academy of Research and Education,Kanchipuram District,Tamil Nadu,39.97,98
98,Kurukshetra University,Kurukshetra,Haryana,39.79,99


In [14]:
top_univ_df = selected_df[['State','University']].groupby('State').agg(['count'])
top_univ_df = top_univ_df.rename(columns = {'University':'State', 'count':'Univ Count'})
top_univ_df.columns.values

array([('State', 'Univ Count')], dtype=object)

In [15]:
top_univ_df.columns

MultiIndex([('State', 'Univ Count')],
           )

In [16]:
file_name = 'top_univ_state.csv'
top_univ_df.to_csv(file_name, sep='|', encoding='utf-8')